<a href="https://colab.research.google.com/github/zeppelin-slash/Hotel-Recommnder/blob/main/Final_Project_Hybrid_Hotel_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn import preprocessing

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Library for Collaborative filtering
!pip install surprise
from surprise import Reader,Dataset,SVD
from surprise.model_selection import cross_validate
import warnings;warnings.simplefilter('ignore')
%matplotlib inline



## **Aim: To build a hotel recommendation system using user preferences, city and a hotel name.**

In [ ]:
hotel_data = pd.read_csv('https://raw.githubusercontent.com/cjflanagan/cs68/master/Datafiniti_Hotel_Reviews%20-%20Datafiniti_Hotel_Reviews.csv')
hotel_data.columns = hotel_data.columns.str.strip()
hotel_data.columns = hotel_data.columns.str.replace(".","_")

In [ ]:
hotel_data.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,reviews_date,reviews_dateSeen,reviews_rating,reviews_sourceURLs,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,sourceURLs,websites
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2013-11-14T00:00:00Z,"2016-08-03T00:00:00Z,2016-07-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
1,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2014-07-06T00:00:00Z,"2016-08-02T00:00:00Z,2016-08-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,NaN,NaN,D,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
2,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2015-01-02T00:00:00Z,"2016-11-15T00:00:00Z,2016-08-23T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,NaN,NaN,Ron,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
3,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-05-15T00:00:00Z,"2016-05-21T00:00:00Z,2016-07-31T00:00:00Z",2.0,https://www.tripadvisor.com/Hotel_Review-g4118...,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,jaeem2016,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...
4,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-07-09T00:00:00Z,2016-07-31T00:00:00Z,5.0,https://www.tripadvisor.com/Hotel_Review-g4118...,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD,MamaNiaOne,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...


In [ ]:
hotel_data['reviews_text']

0       Our experience at Rancho Valencia was absolute...
1       Amazing place. Everyone was extremely warm and...
2       We booked a 3 night stay at Rancho Valencia to...
3       Currently in bed writing this for the past hr ...
4       I live in Md and the Aloft is my Home away fro...
                              ...                        
3964                           Nice hotel/ great location
3965    very nice stay.. will be back to another location
3966    Only stayed one night but that was enough to e...
3967    I have been staying at the Four Points Sherato...
3968    Thank you Erik for taking care of my friends a...
Name: reviews_text, Length: 3969, dtype: object

Content Based Recommendation on Hotel Reviews

In [ ]:
hotel_data.shape

hotel_data['reviews_title'] = hotel_data['reviews_title'].replace(np.nan, '', regex=True)

In [ ]:
#aggregate text data by hotel,city

data = hotel_data.groupby(['id','dateAdded','dateUpdated','address','categories','primaryCategories','city','country','latitude','longitude','name','postalCode','province','keys']).agg(lambda x: ', '.join(list(set(list(x))))).reset_index()

#data = hotel_data.groupby(['city','country','postalCode','province']).agg(lambda x: ', '.join(list(set(list(x))))).reset_index()

In [ ]:
reviews_mean = hotel_data.groupby(['id','dateAdded','dateUpdated','address','categories','primaryCategories','city','country','latitude','longitude','name','postalCode','province','keys'])['reviews_rating'].agg(['mean']).rename({'mean':'reviews_mean'},axis=1)
reviews_count = hotel_data.groupby(['id','dateAdded','dateUpdated','address','categories','primaryCategories','city','country','latitude','longitude','name','postalCode','province','keys'])['reviews_username'].agg(['count']).rename({'count':'reviews_count'},axis=1)
#temp = hotel_data.groupby('card1')['TransactionAmt'].agg(['mean']).rename({'mean':'TransactionAmt_card1_mean'},axis=1)
df = pd.merge(data,reviews_mean,on=['id','dateAdded','dateUpdated','address','categories','primaryCategories','city','country','latitude','longitude','name','postalCode','province','keys'],how='left')
hotel_agg = pd.merge(df,reviews_count,on=['id','dateAdded','dateUpdated','address','categories','primaryCategories','city','country','latitude','longitude','name','postalCode','province','keys'],how='left')
hotel_agg.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,latitude,longitude,name,postalCode,province,keys,reviews_date,reviews_dateSeen,reviews_sourceURLs,reviews_text,reviews_title,reviews_username,sourceURLs,websites,reviews_mean,reviews_count
0,AV-TGsFqRxPSIh2RmVF-,2017-11-06T20:51:12Z,2018-09-04T21:27:52Z,107 Merrimac Street,"Hotels,Hotel",Accommodation & Food Services,Boston,US,42.363911,-71.062400,The Boxer,2114,MA,us/ma/boston/107merrimacstreet/93930424,"2018-04-09T00:00:00.000Z, 2018-03-30T00:00:00....","2018-05-06T00:00:00Z, 2018-04-14T00:00:00Z, 20...",https://www.booking.com/reviews/us/hotel/bulfi...,"Good: The room was very comfortable, great for...","Pleasant stay in this hotel., Classic and Clos...","Paul, Jimmy, Marius, Andrew, Becky, Lisa, Brad...",https://www.telegraph.co.uk/travel/destination...,http://theboxerboston.com/,4.285000,50
1,AV-TGuWBa4HuVbedGcJK,2017-11-06T20:51:11Z,2018-09-04T21:27:43Z,Two East 55th Street at Fifth Avenue,"Hotels,Hotel",Accommodation & Food Services,New York,US,40.761408,-73.974300,The St. Regis New York,10022,NY,us/ny/newyork/twoeast55thstreetatfifthavenue/-...,"2017-11-14T00:00:00.000Z, 2017-06-15T00:00:00....",2018-08-26T00:00:00Z,https://www.tripadvisor.com/Hotel_Review-g6076...,Best place for lunch cocktail's the ole king c...,"Best Hotel in New York, The Best Hotel I've Ev...","BlackCatMN, adeeyacoby, Billy R, kennstransky,...",https://www.tripadvisor.com/Hotel_Review-g6076...,http://www.stregisnewyork.com/,4.529412,51
2,AV-TGwK3IxWefVJwr5IJ,2017-11-06T20:51:11Z,2018-09-04T21:27:39Z,20 West Kinzie Street,"Hotels,Hotel",Accommodation & Food Services,Chicago,US,41.889408,-87.628939,Kinzie Hotel,60654,IL,us/il/chicago/20westkinziestreet/153270062,"2016-09-17T00:00:00.000Z, 2016-07-26T00:00:00....","2018-04-21T00:00:00Z, 2018-03-27T00:00:00Z, 20...",https://www.booking.com/reviews/us/hotel/amalf...,"Good: service was prompt and friendly., Good: ...","Not great for family on Saturday nights, Staff...","Stan, Mike, Saad, Kenneth, James, Derrick, Dav...",https://www.booking.com/reviews/us/hotel/amalf...,http://www.kinziehotel.com/,4.438043,138
3,AV-TGwqGIxWefVJwr5Ji,2017-11-06T20:51:12Z,2018-09-04T21:27:51Z,801 Collins Avenue,"Hotels,Hotel",Accommodation & Food Services,Miami Beach,US,25.778280,-80.131890,The Hotel of South Beach,33139,FL,us/fl/miamibeach/801collinsavenue/-544378751,"2018-01-10T00:00:00.000Z, 2018-03-17T00:00:00....","2018-03-27T00:00:00Z,2018-04-14T00:00:00Z",https://www.booking.com/reviews/us/hotel/south...,Bad: Very noisy at night (24h party directly b...,First time in Miami and had a great stay at Th...,"Marilyn, Amadou, Peter, Karin, Nicole sal, Dub...",https://www.booking.com/reviews/us/hotel/south...,http://www.thehotelofsouthbeach.com/,4.385714,7
4,AV0J-cFz-gnIPe8DU7eF,2017-07-03T19:42:29Z,2018-08-27T16:52:49Z,207 Porter Street,"Wedding Sites,Meeting Facilities,Hotels Motels...",Accommodation & Food Services,Boston,US,42.370400,-71.031600,Embassy Suites by Hilton Boston at Logan Airport,02128-2213,MA,us/ma/boston/207porterstreet/-1882827066,2018-08-24T00:00:00.000Z,2018-08-25T00:00:00Z,https://www.tripadvisor.com/Hotel_Review-g6074...,Nice place with a great staff. This place is c...,Nice place with a 5 minute shuttle ride,Tim L,https://www.tripadvisor.com/Hotel_Review-g6074...,https://ad.doubleclick.net/ddm/clk/317271033;1...,5.000000,1


In [ ]:
data.shape

(478, 22)

In [ ]:
data.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,latitude,longitude,name,postalCode,province,keys,reviews_date,reviews_dateSeen,reviews_sourceURLs,reviews_text,reviews_title,reviews_username,sourceURLs,websites
0,AV-TGsFqRxPSIh2RmVF-,2017-11-06T20:51:12Z,2018-09-04T21:27:52Z,107 Merrimac Street,"Hotels,Hotel",Accommodation & Food Services,Boston,US,42.363911,-71.062400,The Boxer,2114,MA,us/ma/boston/107merrimacstreet/93930424,"2018-04-09T00:00:00.000Z, 2018-03-30T00:00:00....","2018-05-06T00:00:00Z, 2018-04-14T00:00:00Z, 20...",https://www.booking.com/reviews/us/hotel/bulfi...,"Good: The room was very comfortable, great for...","Pleasant stay in this hotel., Classic and Clos...","Paul, Jimmy, Marius, Andrew, Becky, Lisa, Brad...",https://www.telegraph.co.uk/travel/destination...,http://theboxerboston.com/
1,AV-TGuWBa4HuVbedGcJK,2017-11-06T20:51:11Z,2018-09-04T21:27:43Z,Two East 55th Street at Fifth Avenue,"Hotels,Hotel",Accommodation & Food Services,New York,US,40.761408,-73.974300,The St. Regis New York,10022,NY,us/ny/newyork/twoeast55thstreetatfifthavenue/-...,"2017-11-14T00:00:00.000Z, 2017-06-15T00:00:00....",2018-08-26T00:00:00Z,https://www.tripadvisor.com/Hotel_Review-g6076...,Best place for lunch cocktail's the ole king c...,"Best Hotel in New York, The Best Hotel I've Ev...","BlackCatMN, adeeyacoby, Billy R, kennstransky,...",https://www.tripadvisor.com/Hotel_Review-g6076...,http://www.stregisnewyork.com/
2,AV-TGwK3IxWefVJwr5IJ,2017-11-06T20:51:11Z,2018-09-04T21:27:39Z,20 West Kinzie Street,"Hotels,Hotel",Accommodation & Food Services,Chicago,US,41.889408,-87.628939,Kinzie Hotel,60654,IL,us/il/chicago/20westkinziestreet/153270062,"2016-09-17T00:00:00.000Z, 2016-07-26T00:00:00....","2018-04-21T00:00:00Z, 2018-03-27T00:00:00Z, 20...",https://www.booking.com/reviews/us/hotel/amalf...,"Good: service was prompt and friendly., Good: ...","Not great for family on Saturday nights, Staff...","Stan, Mike, Saad, Kenneth, James, Derrick, Dav...",https://www.booking.com/reviews/us/hotel/amalf...,http://www.kinziehotel.com/
3,AV-TGwqGIxWefVJwr5Ji,2017-11-06T20:51:12Z,2018-09-04T21:27:51Z,801 Collins Avenue,"Hotels,Hotel",Accommodation & Food Services,Miami Beach,US,25.778280,-80.131890,The Hotel of South Beach,33139,FL,us/fl/miamibeach/801collinsavenue/-544378751,"2018-01-10T00:00:00.000Z, 2018-03-17T00:00:00....","2018-03-27T00:00:00Z,2018-04-14T00:00:00Z",https://www.booking.com/reviews/us/hotel/south...,Bad: Very noisy at night (24h party directly b...,First time in Miami and had a great stay at Th...,"Marilyn, Amadou, Peter, Karin, Nicole sal, Dub...",https://www.booking.com/reviews/us/hotel/south...,http://www.thehotelofsouthbeach.com/
4,AV0J-cFz-gnIPe8DU7eF,2017-07-03T19:42:29Z,2018-08-27T16:52:49Z,207 Porter Street,"Wedding Sites,Meeting Facilities,Hotels Motels...",Accommodation & Food Services,Boston,US,42.370400,-71.031600,Embassy Suites by Hilton Boston at Logan Airport,02128-2213,MA,us/ma/boston/207porterstreet/-1882827066,2018-08-24T00:00:00.000Z,2018-08-25T00:00:00Z,https://www.tripadvisor.com/Hotel_Review-g6074...,Nice place with a great staff. This place is c...,Nice place with a 5 minute shuttle ride,Tim L,https://www.tripadvisor.com/Hotel_Review-g6074...,https://ad.doubleclick.net/ddm/clk/317271033;1...


In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

Missing Data check

In [ ]:
#missing data
total = data.isna().sum().sort_values(ascending=False) 
percent = (data.isna().sum()/data.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(50)

,Total,Percent
websites,0,0.0
sourceURLs,0,0.0
dateAdded,0,0.0
dateUpdated,0,0.0
address,0,0.0
categories,0,0.0
primaryCategories,0,0.0
city,0,0.0
country,0,0.0
latitude,0,0.0


Cleaning

In [ ]:
data.columns = data.columns.str.strip()
data.columns = data.columns.str.replace(".","_")
data.tail()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,latitude,longitude,name,postalCode,province,keys,reviews_date,reviews_dateSeen,reviews_sourceURLs,reviews_text,reviews_title,reviews_username,sourceURLs,websites
473,AWKpYhaja4HuVbedPVes,2018-04-09T07:13:49Z,2018-09-04T21:27:48Z,350 Ocean Drive,"Hotels,Hotel",Accommodation & Food Services,Miami Beach,US,25.773148,-80.13260,Room Mate Lord Balfour,33139,FL,us/fl/miamibeach/350oceandrive/207308734,"2017-12-20T00:00:00.000Z, 2018-03-30T00:00:00....","2018-03-28T00:00:00Z, 2018-03-28T00:00:00Z,201...",https://www.booking.com/reviews/us/hotel/lord-...,Bad: As others have commented the Valet Parkin...,"can't wait to stay here again!, Bed Bugs, Ocea...","John, Kathleen, Robert, Iris, Elizabeth, David...",https://www.booking.com/reviews/us/hotel/lord-...,http://lordbalfour.room-matehotels.com/
474,AWKr8UBHa4HuVbedPWl2,2018-04-09T18:52:29Z,2018-09-04T21:27:41Z,328 East 1st Street,"Hotels,Hotel",Accommodation & Food Services,Los Angeles,US,34.049671,-118.24019,Miyako Hotel Los Angeles,90012,CA,us/ca/losangeles/328east1ststreet/-1050079029,"2016-08-31T00:00:00.000Z, 2016-09-23T00:00:00....","2018-03-28T00:00:00Z, 2018-03-27T00:00:00Z, 20...",https://www.booking.com/reviews/us/hotel/miyak...,Bad: Happy hour prices to expensive. Good: Loc...,"small fridge coffee maker, Great for Ramen ent...","Jerry, Tina, Jayde, Nancy, Laura, Alan, Lauren...",https://www.booking.com/hotel/us/miyako-los-an...,http://www.miyakoinn.com
475,AWKuEeWka4HuVbedPXff,2018-04-10T05:14:23Z,2018-09-04T21:27:39Z,630 North Rush Street,"Hotels,Hotel",Accommodation & Food Services,Chicago,US,41.893535,-87.62563,AC Hotel Chicago Downtown,60611,IL,us/il/chicago/630northrushstreet/-813812852,"2017-03-13T00:00:00.000Z, 2017-02-08T00:00:00....","2018-05-06T00:00:00Z, 2018-03-28T00:00:00Z,201...",https://www.booking.com/reviews/us/hotel/rush....,Bad: Me and my husband came to Chicago for our...,"Wonderful stay!, Good Hotel Few Problems, The ...","Meagan, Bradley, John, Yolanda, Krist√≠n, Mari...",https://www.booking.com/reviews/us/hotel/rush....,http://www.marriott.com/hotels/travel/chiac-ac...
476,AWNSHai33-Khe5l_ifzh,2018-05-12T02:09:00Z,2018-09-04T21:27:50Z,129 Paoakalani Avenue,"Hotels,Hotel",Accommodation & Food Services,Honolulu,US,21.272780,-157.82226,Hotel Renew,96815,HI,us/hi/honolulu/129paoakalaniavenue/924924249,"2007-07-07T00:00:00.000Z, 2007-02-07T00:00:00....","2018-08-24T00:00:00Z, 2018-08-27T00:00:00Z",https://www.tripadvisor.com/Hotel_Review-g6098...,"When you read the information about Lenore, be...","Great stay experience!, Friendly staff, great ...","jenebean, mochiball, dookie1902, nmc75, Travel...",https://www.tripadvisor.com/Hotel_Review-g6098...,http://www.hotelrenew.com/
477,AWUPNmq6IxWefVJw3c71,2018-08-06T12:11:05Z,2018-09-04T21:27:39Z,530 N LaSalle Dr,"Hotels,Hotel",Accommodation & Food Services,Chicago,US,41.891755,-87.63284,Moxy Chicago Downtown,60654,IL,us/il/chicago/530nlasalledr/-591264229,2018-08-24T00:00:00.000Z,2018-08-25T00:00:00Z,https://www.tripadvisor.com/Hotel_Review-g3580...,"Stayed a few days while on business, big mista...",Not for Business Travel,DLama,https://www.tripadvisor.com/Hotel_Review-g3580...,https://www.marriott.com/hotels/travel/chiox-m...


Process Reviews

In [ ]:
data['reviews_text'] = data['reviews_text'].replace(np.nan, '', regex=True)
tfidf_matrix = tfidf.fit_transform(data['reviews_text'])
tfidf_matrix.shape

(478, 10522)

In [ ]:
tfidf_matrix

<478x10522 sparse matrix of type '<class 'numpy.float64'>'
	with 66930 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.18603629, 0.67962979, ..., 0.3999553 , 0.27874103,
        0.09303662],
       [0.18603629, 1.        , 0.2116688 , ..., 0.10533665, 0.24770492,
        0.08332678],
       [0.67962979, 0.2116688 , 1.        , ..., 0.46147939, 0.31291923,
        0.11929109],
       ...,
       [0.3999553 , 0.10533665, 0.46147939, ..., 1.        , 0.18464087,
        0.08465337],
       [0.27874103, 0.24770492, 0.31291923, ..., 0.18464087, 1.        ,
        0.08108403],
       [0.09303662, 0.08332678, 0.11929109, ..., 0.08465337, 0.08108403,
        1.        ]])

In [ ]:
pd.DataFrame(cosine_sim, columns=data.name.values, index=data.name.values)

,The Boxer,The St. Regis New York,Kinzie Hotel,The Hotel of South Beach,Embassy Suites by Hilton Boston at Logan Airport,Courtyard Austin South,Aviator Hotel Anchorage,Fairfield Inn and Suites by Marriott Virginia Beach Oceanfront,Pine Barn Inn,Hampton Inn and Suites Baltimore Inner Harbor,Homewood Suites by Hilton Baltimore,Hampton Inn and Suites Williamsburg-Central,Hilton Garden Inn Williamsburg,Central Park Hotel,American Inn,Americas Best Value Inn & Suites Canon City,Aloft Philadelphia Downtown,Bricco Suites,The James New York ‚Äì NoMad,Candlewood Suites Somerset,Chalet Motel,Embassy Suites by Hilton Philadelphia - Center City,"Pier 5 Hotel Baltimore, Curio Collection by Hilton",The Westin Chicago River North,Pacific Inn,Hampton Inn & Suites Altoona-des Moines,The Pearl Hotel,Hampton Inn & Suites St. Louis/South I-55,Quality Inn Midway Airport,Hampton Inn & Suites Bradenton Downtown Historic District,SpringHill Suites by Marriott Virginia Beach Oceanfront,Fairfield Inn & Suites by Marriott Key West at The Keys Collection,Central Park West Hostel,Quality Inn & Suites,Sheraton Portland Airport Hotel,Top's Motel,Comfort Inn I-10 West at 51st Ave,Hyatt Regency-valencia,Cooper Hill Inn,Honua Kai Resort Spa,...,Hotel St. Augustine,Montage Beverly Hills,Sense Beach House,Hyatt Regency Sacramento,W Boston,Virgin Hotels Chicago,Royal Resort,Bienville House,WoodSpring Suites Phoenix I-10 West,Days Inn Baltimore South/Glen Burnie,Maison de Macarty,Best Western Envoy Inn,Embassy Suites by Hilton Baltimore at BWI Airport,Drury Inn St Louis Collinsville,Hampton Inn Suites National HarborAlexandria Area,MOXY New Orleans Downtown/French Quarter Area,Four Seasons Hotel New York Downtown,Ace Hotel Chicago,Hilton Baltimore BWI Airport,Quality Inn & Suites Anaheim Maingate,Comfort Suites,"The Troubadour Hotel New Orleans, Tapestry Collection by Hilton",AC Hotel by Marriott Boston Downtown,dana hotel and spa,Suncoast Hotel and Casino,DoubleTree by Hilton Hotel Washington DC - Silver Spring,Extended Stay America - Atlanta - Clairmont,Holiday Inn Express Baltimore-Bwi Airport West,Rodeway Inn,Hilton Garden Inn BWI Airport,Hyatt Place Baltimore/Inner Harbor,The Westin Las Vegas Hotel & Spa,Fremont Hotel & Casino,Hotel Zetta San Francisco,Main Street Station Casino Brewery Hotel,Room Mate Lord Balfour,Miyako Hotel Los Angeles,AC Hotel Chicago Downtown,Hotel Renew,Moxy Chicago Downtown
The Boxer,1.000000,0.186036,0.679630,0.427397,0.072047,0.139014,0.089249,0.458345,0.066176,0.528266,0.552562,0.500935,0.473758,0.104865,0.089976,0.095332,0.255894,0.105577,0.042647,0.095526,0.027400,0.085337,0.414896,0.481797,0.020558,0.103735,0.152973,0.164895,0.204457,0.091702,0.555001,0.240239,0.117594,0.095235,0.294749,0.058319,0.109312,0.162428,0.047976,0.159622,...,0.566018,0.397541,0.534822,0.080132,0.670977,0.482269,0.163117,0.052577,0.089124,0.284426,0.097468,0.268229,0.283665,0.133226,0.306983,0.087247,0.080140,0.434440,0.286865,0.092311,0.249260,0.260921,0.195179,0.592459,0.277434,0.353158,0.121581,0.251433,0.173875,0.337274,0.361850,0.648480,0.610598,0.643281,0.560661,0.624001,0.492331,0.399955,0.278741,0.093037
The St. Regis New York,0.186036,1.000000,0.211669,0.142950,0.051314,0.121880,0.022877,0.140256,0.064599,0.134746,0.123421,0.117339,0.122639,0.043268,0.049547,0.063330,0.289071,0.061658,0.054244,0.076542,0.020905,0.060585,0.213353,0.129918,0.026903,0.094165,0.105045,0.123664,0.085405,0.079006,0.173436,0.347595,0.074286,0.177024,0.103897,0.051094,0.095185,0.133454,0.019158,0.138843,...,0.140416,0.155216,0.155234,0.041749,0.183050,0.192626,0.068006,0.064424,0.152490,0.197484,0.131832,0.184026,0.226862,0.059792,0.191912,0.096592,0.106368,0.078804,0.242420,0.106473,0.168156,0.433694,0.048468,0.184793,0.226260,0.213399,0.231659,0.190989,0.063975,0.223147,0.220693,0.173178,0.157190,0.196903,0.128352,0.154310,0.127430,0.105337,0.247705,0.083327
Kinzie Hotel,0.679630,0.211669,1.000000,0.449140,0.113785,0.153626,0.093445,0.534532,0.042346,0.630440,0.67285

In [ ]:
indices = pd.Series(data.index, index=data.name).drop_duplicates()
indices

name
The Boxer                                             0
The St. Regis New York                                1
Kinzie Hotel                                          2
The Hotel of South Beach                              3
Embassy Suites by Hilton Boston at Logan Airport      4
                                                   ... 
Room Mate Lord Balfour                              473
Miyako Hotel Los Angeles                            474
AC Hotel Chicago Downtown                           475
Hotel Renew                                         476
Moxy Chicago Downtown                               477
Length: 478, dtype: int64

In [ ]:
idx = indices['Moxy Chicago Downtown']
idx

477

In [ ]:
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores

[(0, 0.09303662320737813),
 (1, 0.08332678353951299),
 (2, 0.11929108566897169),
 (3, 0.03995356392355642),
 (4, 0.024555053893198456),
 (5, 0.006335804293511518),
 (6, 0.0),
 (7, 0.0820193323695961),
 (8, 0.018957033318610237),
 (9, 0.051983003762280065),
 (10, 0.07088359037218857),
 (11, 0.0516425794142789),
 (12, 0.07236093382967831),
 (13, 0.008881030565016025),
 (14, 0.04850502577320111),
 (15, 0.017773339943662165),
 (16, 0.08623365462444485),
 (17, 0.01145673391460737),
 (18, 0.01074715008004615),
 (19, 0.04349370153192391),
 (20, 0.005580629416688474),
 (21, 0.05013648812020299),
 (22, 0.0865430980556469),
 (23, 0.05409779171687673),
 (24, 0.0020520137284097397),
 (25, 0.013102098665310839),
 (26, 0.04079511720563558),
 (27, 0.02293029034832332),
 (28, 0.06205559679169966),
 (29, 0.019503687611267317),
 (30, 0.05218233429161126),
 (31, 0.07509013277407002),
 (32, 0.028727044054719784),
 (33, 0.06455274215820328),
 (34, 0.057025714007088335),
 (35, 0.027809612368926998),
 (36, 0

In [ ]:
sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)

In [ ]:
sim_scores = sim_scores[1:11]
sim_scores

[(188, 0.15550362989623337),
 (228, 0.13293397343243085),
 (450, 0.12838334738238932),
 (447, 0.12570013622112405),
 (88, 0.12470151710160804),
 (248, 0.12070194314966919),
 (2, 0.11929108566897169),
 (463, 0.11740146609567163),
 (395, 0.11549619569833129),
 (389, 0.1135833417247644)]

In [ ]:
hotel_indices = [i[0] for i in sim_scores]
hotel_indices

[188, 228, 450, 447, 88, 248, 2, 463, 395, 389]

In [ ]:
# Function that takes in name as input and outputs most similar hotels
def get_recommendations(name, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[name]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the hotel indices
    hotel_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return hotel_agg['name'].iloc[hotel_indices]

In [ ]:
get_recommendations('River Hotel')

228                                   Central Loop Hotel
213                       InterContinental San Francisco
2                                           Kinzie Hotel
468                   Hyatt Place Baltimore/Inner Harbor
459    The Troubadour Hotel New Orleans, Tapestry Col...
467                        Hilton Garden Inn BWI Airport
338                                   Omni Royal Orleans
476                                          Hotel Renew
463    DoubleTree by Hilton Hotel Washington DC - Sil...
456                         Hilton Baltimore BWI Airport
Name: name, dtype: object

# **City Based Recommendation**

In [ ]:
review_counts = hotel_agg[hotel_agg['reviews_count'].notnull()]['reviews_count'].astype('float')
review_averages = hotel_agg[hotel_agg['reviews_mean'].notnull()]['reviews_mean'].astype('float')
C = review_averages.mean()
C

3.998570839178892

In [ ]:
m = review_counts.quantile(0.95)
m

39.14999999999998

In [ ]:
qualified = hotel_agg[(hotel_agg['reviews_count'] >= m) & (hotel_agg['reviews_count'].notnull()) & (hotel_agg['reviews_mean'].notnull())][['id','dateAdded','dateUpdated','address','categories','primaryCategories','city','country','latitude','longitude','name','postalCode','province','keys','reviews_count','reviews_mean']]
qualified['reviews_count'] = qualified['reviews_count'].astype('float')
qualified['reviews_mean'] = qualified['reviews_mean'].astype('float')
qualified.shape

(24, 16)

In [ ]:
def weighted_rating(x):
    v = x['reviews_count']
    R = x['reviews_mean']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [ ]:
qualified.head(15)

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,latitude,longitude,name,postalCode,province,keys,reviews_count,reviews_mean,wr
233,AVwdMsswkufWRAb53Vy0,2017-04-10T07:43:42Z,2018-06-29T19:35:18Z,3730 Las Vegas Blvd S,"Hotels,Casinos,Resorts,Resort,Casino,Hotel",Accommodation & Food Services,Las Vegas,US,36.107741,-115.176268,ARIA Resort Casino,89158,NV,us/nv/lasvegas/3730lasvegasblvds/-1699935985,139.0,4.503597,4.392613
2,AV-TGwK3IxWefVJwr5IJ,2017-11-06T20:51:11Z,2018-09-04T21:27:39Z,20 West Kinzie Street,"Hotels,Hotel",Accommodation & Food Services,Chicago,US,41.889408,-87.628939,Kinzie Hotel,60654,IL,us/il/chicago/20westkinziestreet/153270062,138.0,4.438043,4.340920
1,AV-TGuWBa4HuVbedGcJK,2017-11-06T20:51:11Z,2018-09-04T21:27:43Z,Two East 55th Street at Fifth Avenue,"Hotels,Hotel",Accommodation & Food Services,New York,US,40.761408,-73.974300,The St. Regis New York,10022,NY,us/ny/newyork/twoeast55thstreetatfifthavenue/-...,51.0,4.529412,4.298880
39,AVwc-UVS_7pvs4fz07J4,2017-04-18T11:08:50Z,2018-05-28T16:11:26Z,130 Kai Malina Pkwy,"Hotels,Resort,Spa,Hotel,Hotel Pool",Accommodation & Food Services,Lahaina,US,20.944290,-156.690786,Honua Kai Resort Spa,96761,HI,us/hi/lahaina/130kaimalinapkwy/657884627,99.0,4.404040,4.289135
338,AVwdygbvIN2L1WUfzOvF,2017-04-18T10:57:07Z,2018-06-26T03:04:02Z,621 Saint Louis St,"Hotels,Resort,Lodging,Hotel",Accommodation & Food Services,New Orleans,US,29.956219,-90.065894,Omni Royal Orleans,70130,LA,us/la/neworleans/621saintlouisst/505139068,44.0,4.500000,4.263909
293,AVwdh6K4ByjofQCxqA0x,2016-11-05T00:16:03Z,2018-06-22T18:18:47Z,2241 Price St,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Pismo Beach,US,35.147960,-120.649605,Seacrest Oceanfront Hotel,93449,CA,us/ca/pismobeach/2241pricest/-1866149919,41.0,4.512195,4.261311
461,AWEw3H-UIxWefVJwyAZ0,2018-01-26T05:06:32Z,2018-09-04T21:27:39Z,660 North State Street,"Hotels,Hotel",Accommodation & Food Services,Chicago,US,41.894240,-87.628370,dana hotel and spa,60654,IL,us/il/chicago/660northstatestreet/-2138362272,46.0,4.482609,4.260059
468,AWIajIQMa4HuVbedO4uB,2018-03-12T14:02:50Z,2018-05-28T16:09:53Z,511 S Central Ave,"Hotels,Hotels and motels,Hotel",Accommodation & Food Services,Baltimore,US,39.284641,-76.599013,Hyatt Place Baltimore/Inner Harbor,21202,MD,us/md/baltimore/511scentralave/637315586,54.0,4.388889,4.224842
467,AWIYGr06IxWefVJwzU-7,2018-03-12T01:20:57Z,2018-05-28T16:18:29Z,1516 Aero Dr,"Hotels,Hotel",Accommodation & Food Services,Linthicum Heights,US,39.210072,-76.677158,Hilton Garden Inn BWI Airport,21090,MD,us/md/linthicumheights/1516aerodr/7992643,41.0,4.439024,4.223881
471,AWKoW_tYa4HuVbedPVUg,2018-04-09T02:32:32Z,2018-09-04T21:27:39Z,55 5th Street,"Hotels,Hotel",Accommodation & Food Services,San Francisco,US,37.783529,-122.406860,Hotel Zetta San Francisco,94103,CA,us/ca/sanfrancisco/555thstreet/-67856798,63.0,4.350794,4.215801


In [ ]:
s = hotel_agg.apply(lambda x: pd.Series(x['city']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'city'
gen_md = hotel_agg.drop('city', axis=1).join(s)

In [ ]:
gen_md.shape

(478, 24)

In [ ]:
def build_chart(city, percentile=0.85):
    df = gen_md[gen_md['city'] == city]
    review_counts = hotel_agg[hotel_agg['reviews_count'].notnull()]['reviews_count'].astype('float')
    review_averages = hotel_agg[hotel_agg['reviews_mean'].notnull()]['reviews_mean'].astype('float')
    C = review_averages.mean()
    m = review_counts.quantile(percentile)
    
    qualified = df[(df['reviews_count'] >= m) & (df['reviews_count'].notnull()) & (df['reviews_mean'].notnull())][['id','address','name','reviews_count','reviews_mean']]
    qualified['reviews_count'] = qualified['reviews_count'].astype('float')
    qualified['reviews_mean'] = qualified['reviews_mean'].astype('float')
    
    qualified['wr'] = qualified.apply(lambda x: (x['reviews_count']/(x['reviews_count']+m) * x['reviews_mean']) + (m/(m+x['reviews_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [ ]:
build_chart('Chicago').head(5)

,id,address,name,reviews_count,reviews_mean,wr
443,AWE2CYSsIxWefVJwyDrn,203 N Wabash Ave,Virgin Hotels Chicago,19.0,4.776316,4.468433
2,AV-TGwK3IxWefVJwr5IJ,20 West Kinzie Street,Kinzie Hotel,138.0,4.438043,4.401676
461,AWEw3H-UIxWefVJwyAZ0,660 North State Street,dana hotel and spa,46.0,4.482609,4.379507
23,AV1ha6JpIxWefVJwgDSa,320 North Dearborn Street,The Westin Chicago River North,14.0,4.592857,4.313127
228,AVwdLbuu_7pvs4fz3FIK,111 W Adams St,Central Loop Hotel,20.0,4.250000,4.153535


# **Collabartive Filtering using SVD**

In [ ]:
ratings = hotel_data[['reviews_username', 'name', 'reviews_rating']]
ratings.head()

,reviews_username,name,reviews_rating
0,Paula,Rancho Valencia Resort Spa,5.0
1,D,Rancho Valencia Resort Spa,5.0
2,Ron,Rancho Valencia Resort Spa,5.0
3,jaeem2016,Aloft Arundel Mills,2.0
4,MamaNiaOne,Aloft Arundel Mills,5.0


In [ ]:
reader = Reader()



# get just top 100K rows for faster run time
data = Dataset.load_from_df(ratings[['reviews_username', 'name', 'reviews_rating']], reader)
#data.split(n_folds=3)

svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'fit_time': (0.21581578254699707,
  0.2210392951965332,
  0.21129369735717773,
  0.21054983139038086,
  0.21865510940551758),
 'test_mae': array([0.78206842, 0.75406602, 0.79745693, 0.78415646, 0.78511846]),
 'test_rmse': array([0.98918815, 0.96393348, 1.03986412, 1.01394449, 1.02242915]),
 'test_time': (0.009153127670288086,
  0.0064733028411865234,
  0.006524085998535156,
  0.015334844589233398,
  0.00957179069519043)}

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
ratings[ratings['reviews_username'] == 'A verified traveler']

,reviews_username,name,reviews_rating
28,A verified traveler,Hampton Inn Suites National HarborAlexandria Area,4.0
29,A verified traveler,Hampton Inn Suites National HarborAlexandria Area,5.0
30,A verified traveler,Hampton Inn Suites National HarborAlexandria Area,2.0
32,A verified traveler,Hampton Inn Suites National HarborAlexandria Area,4.0
34,A verified traveler,Hampton Inn Suites National HarborAlexandria Area,4.0
...,...,...,...
3955,A verified traveler,Embassy Suites by Hilton Baltimore at BWI Airport,5.0
3957,A verified traveler,Embassy Suites by Hilton Baltimore at BWI Airport,4.0
3958,A verified traveler,Embassy Suites by Hilton Baltimore at BWI Airport,4.0
3959,A verified traveler,Embassy Suites by Hilton Baltimore at BWI Airport,2.0


In [ ]:
#hotel_data['reviews_username'].value_counts()

A verified traveler    311
Anonymous              155
A Traveler              97
John                    22
David                   20
                      ... 
Sarit C                  1
Hope                     1
TruthDIVA                1
MNWinterBlues            1
Teresita M               1
Name: reviews_username, Length: 2918, dtype: int64

In [ ]:
svd.predict('David', 'dana hotel and spa')

Prediction(uid='David', iid='dana hotel and spa', r_ui=None, est=4.526229854637817, details={'was_impossible': False})

# **Hybrid Classifier**

In [ ]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
id_map = hotel_agg[['name']]

id_map = id_map.merge(hotel_agg[['name', 'id']], on='name').set_index('name')
id_map = id_map.set_index('name')
id_map

KeyError: ignored

In [ ]:
indices_map = id_map.set_index('name')

KeyError: ignored

In [ ]:
def hybrid(username, hotel):
    idx = indices[hotel]
    

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    hotel_indices = [i[0] for i in sim_scores]
    
    hotels = hotel_agg.iloc[hotel_indices][['name', 'reviews_count', 'reviews_mean']]
    hotels['est'] = hotels['name'].apply(lambda x: svd.predict(username, sim_scores.loc[idx]['name']).est)
    hotels = hotels.sort_values('est', ascending=False)
    return hotels.head(5)

    hotel_agg.loc()

In [ ]:
idx = indices['River Hotel']
sim_scores = list(enumerate(cosine_sim[int(idx)]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:11]
hotel_indices = [i[0] for i in sim_scores]

In [ ]:
hotel_indices

[228, 213, 2, 468, 459, 467, 338, 476, 463, 456]

In [ ]:
hotels=hotel_agg.iloc[hotel_indices][['name', 'reviews_count', 'reviews_mean']]
hotels.head()

NameError: ignored

In [ ]:
hotels['est'] = hotels['name'].apply(lambda x: svd.predict('John', simscores.loc[hotel_indices]['name']).est)

NameError: ignored

In [ ]:
hybrid('TruthDIVA', 'River Hotel')

AttributeError: ignored